In [56]:
# c:\doit\ 폴더가 작업 폴더, c:\doit\data\ 폴더가 데이터 저장 폴더
# 12차시 완성 파일에  각 게시글의 내용을 가져오는 파일
# 13, 14차시 통합 검색해서 나온 페이지에서 각 링크를 클릭해서 내용까지 출력


## Step 0. 필요한 모듈과 라이브러리를 로딩하고 검색어를 입력 받습니다

from bs4 import BeautifulSoup     
from selenium import webdriver
import time
import sys
import os
import random
import re
import math
import numpy
import pandas as pd
import xlwt



In [57]:
##Step 2. 사용자에게 검색어 키워드를 입력 받습니다.
print("=" *80)
print("대한민국 구석구석 사이트의 쿼리 정보 받아 정보 수집하기")
print("=" *80)

#query_txt = input('크롤링할 키워드는 무엇입니까?: ')
query_txt = '가을 여행'

# 크롤링 건수와 페이지 확인
#cnt = int(input('크롤링할 건수는 몇건입니까? : '))
cnt = 15
page_cnt = math.ceil( cnt / 10)

# 저장 폴더 생성 및 파일명 생성

#f_dir = input("결과 파일을 저장할 폴더명만 쓰세요(없으면 :c:\\doit\\data\\):")

if f_dir == '':
    f_dir = "C:\\doit\\data\\"
    
# 저장될 파일위치와 이름을 자동 지정합니다

now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

ff_name = f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fx_name = f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'
fc_name = f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'

#ff_name = input('검색 결과를 저장할 txt 파일경로와 이름을 지정하세요(예:c:\\temp\\test.txt): ')
#fc_name = input('검색 결과를 저장할 csv 파일경로와 이름을 지정하세요(예:c:\\temp\\test.csv): ')
#fx_name = input('검색 결과를 저장할 xls 파일경로와 이름을 지정하세요(예:c:\\temp\\test.xls): ')

print( '%s 으로 저장됩니다.\n%s 으로 저장됩니다.\n%s 으로 저장됩니다.\n' %(ff_name, fc_name, fx_name ) )


대한민국 구석구석 사이트의 쿼리 정보 받아 정보 수집하기
C:\doit\data\2020-09-29-16-22-가을 여행\2020-09-29-16-22-가을 여행.txt 으로 저장됩니다.
C:\doit\data\2020-09-29-16-22-가을 여행\2020-09-29-16-22-가을 여행.csv 으로 저장됩니다.
C:\doit\data\2020-09-29-16-22-가을 여행\2020-09-29-16-22-가을 여행.xls 으로 저장됩니다.



In [58]:
##Step 3. 크롬 드라이버를 사용해서 웹 브라우저를 실행합니다.

s_time = time.time() # 시작 시간 설정

path = "c:/doit/chromedriver_win32/chromedriver.exe"
driver = webdriver.Chrome(path)

driver.get("https://korean.visitkorea.or.kr/main/main.html")

# time.sleep(2)  # 창이 모두 열릴 때 까지 2초 기다립니다.
time.sleep(random.randrange(2,5)) # 2 - 5초 사이에 랜덤으로 시간 선택

#코로나 안내 팝업창 있을 경우 닫기 클릭하기 
try :
    driver.find_element_by_xpath('//*[@id="safetyStay1"]/div/div/div/button').click()
except :
    print("안내 팝업창이 없습니다")
    
##Step 4. 검색창의 이름을 찾아서 검색어를 입력합니다

#driver.find_element_by_id("btnSearch").click()  ## 검색 디폴트값에 따라 움직이는데 예외처리 방법으로 해결?

driver.find_element_by_id("btnSearch").click()

element = driver.find_element_by_id("inp_search")

try :
    element.send_keys(query_txt)  # 검색 단추가 없어서 에러나면 다시 한번 눌러서 검색창을 띠움
except:
    driver.find_element_by_id("btnSearch").click()
    element = driver.find_element_by_id("inp_search")
    element.send_keys(query_txt)

driver.find_element_by_link_text("검색").click()

# 위 내용은 '검색어'로 나온 페이지를 
# 14강 원본은 여행지를 찾는 소스(제주도) , 여행지 선택후 브라우저 

#driver.find_element_by_xpath('''//*[@id="btnMenu"]''').click()
#driver.find_element_by_link_text('''여행지''').click()
#time.sleep(3)
#driver.find_element_by_xpath('''//*[@id="39"]/a''').click()
## 여기까지. 

In [59]:
## Step 4. 현재 페이지에 있는 내용을 화면에 출력하기

# 시간 계산을 위한 타임스탬프 찍기
#now1 = time.localtime()
#s1 = '%04d-%02d-%02d-%02d-%02d' % (now1.tm_year, now1.tm_mon, now1.tm_mday, now1.tm_hour, now1.tm_min)
#time.sleep(1)

# 요청건수와 검색건수에 맞추 검색한 후 저장하기 

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
#content_list = soup.find('ul',class_='list_thumType flnon') # 검색 목록으로 나온 화면

result_cnt= soup.find('div', class_='total_check').find('span').get_text( )
result_cnt = result_cnt.replace(',', '')
search_cnt = int(result_cnt)

if cnt > search_cnt :
    cnt = search_cnt
    
print('전체 검색 결과 건수 : ', search_cnt, '건')
print('실제 최종 출력 건수 : ', cnt, '건')
print('\n')

page_cnt = math.ceil(cnt / 10)
print('크롤링 할 총 페이지 번호 : ', page_cnt)
#print('='*80)
#print(content_list)

전체 검색 결과 건수 :  1152 건
실제 최종 출력 건수 :  15 건


크롤링 할 총 페이지 번호 :  2


In [60]:
type(content_list)

bs4.element.Tag

In [55]:
# 추출할 게시글 목록 만들기 

no2 = [ ]
item2 = [ ]
tags2 = [ ]
contents = [ ]

no = 1
no1 = 1
click_cnt = 1
#con_1 = ''
con_1 = []

# 검색 목록으로 나온 화면

for i in range(1, page_cnt+1) :
    content_list = soup.find('ul',class_='list_thumType flnon')
#    con_1 = con_1 + content_list  # 문자열은 태그없는 문자열만 합칠수 있음
    
    if click_cnt > page_cnt+1 :
        break
        
    click_cnt += 1
    
    if (click_cnt % 5 == 0 ):
        driver.find_element_by_link_text('다음').click()
        
    else :
        driver.find_element_by_link_text('''%s''' % click_cnt).click()
        

print (con_1)
# 해당 페이지 화면의 번호, 제목, 태그를 추출


WebDriverException: Message: chrome not reachable
  (Session info: chrome=85.0.4183.121)


In [ ]:
'''
for x in range(1, page_cnt+1) :

    for i in content_list:
        no2.append(no)
        print('번호:',no)
    
        item = i.find('div','tit').get_text( )
        item2.append(item)
        print('제목:',item.strip())
    
        tag = i.find('p','tag').get_text()
        tags2.append(tag)
        print('태그:',tag.strip())
        print("\n")
    
        no += 1

        if no > cnt :
            break
    if click_cnt > page_cnt+1 :
        break
        
    click_cnt += 1
    
    if (click_cnt % 5 == 0 ):
        driver.find_element_by_link_text('다음').click()
        
    else :
        driver.find_element_by_link_text('%s' % click_cnt).click()
        
    time.sleep(2) 



# 해당 페이지에서 각 리스트 세부 내역을 출력하는 내용
# 아래 내용에 위 번호, 제목, 태그 등이 있으니 해당 페이지에서 추출해도 될 듯.


no1 = 1

for x in range(1, page_cnt+1):
    print(' %s 페이지 내용 수집시작합니다.' %x,'='*20, '\n\n')
    
    for i in range(1,11):
        
        if no1 > cnt :
            break
        
        driver.find_element_by_xpath('//*[@id="contents"]/div/div[1]/ul/li[%s]/div[2]/div/a' %i).click()
        time.sleep(2)
       
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        content_list = soup.find('div','box_txtPhoto').find_all('div','txt_p')
        
        for i in content_list :
            con_1 = con_1 + i.text.strip()
        
        print( no1 , ': ', con_1)
        print('\n')
                
        no2.append(no1)
        contents.append(con_1)
        
        driver.back()
        time.sleep(2)
        
        no1 +=1
        
    if click_cnt > page_cnt+1 :
        break
        
    click_cnt += 1
    
    if (click_cnt % 5 == 0 ):
        driver.find_element_by_link_text('다음').click()
        
    else :
        driver.find_element_by_link_text('%s' % click_cnt).click()
        
    time.sleep(2) 
        

    '''

In [80]:
# 소주제 2: 분리 수집된 데이터를 데이터 프레임으로 만들어서 csv , xls 형식으로 저장하기
# 출력 결과를 txt 파일로 저장하기
f = open(ff_name, 'a',encoding='UTF-8')
f.write(str(item2))
f.write(str(tags2))
f.close( )
print(" txt 파일 저장 경로: %s" %ff_name)    
    
# 출력 결과를 표(데이터 프레임) 형태로 만들기
import pandas as pd

korea_trip = pd.DataFrame()
korea_trip['번호']=no2
korea_trip['제목']=item2
korea_trip['내용']=contents
korea_trip['태그']=tags2

# csv 형태로 저장하기
korea_trip.to_csv(fc_name,encoding="utf-8-sig")
print(" csv 파일 저장 경로: %s" %fc_name)

# 엑셀 형태로 저장하기
import xlwt   # pip install xlwt 실행 후 수행
korea_trip.to_excel(fx_name)
print(" xls 파일 저장 경로: %s" %fx_name)

 txt 파일 저장 경로: C:\doit\data\2020-09-28-14-38-가을 여행\2020-09-28-14-38-가을 여행.txt


ValueError: Length of values does not match length of index

In [16]:
# c:\doit\ 폴더가 작업 폴더, c:\doit\data\ 폴더가 데이터 저장 폴더

## Step 0. 필요한 모듈과 라이브러리를 로딩하고 검색어를 입력 받습니다

from bs4 import BeautifulSoup     
from selenium import webdriver
import time
import sys
import os
import re
import math
import numpy
import pandas as pd
import xlwt

##Step 2. 사용자에게 검색어 키워드를 입력 받습니다.
print("=" *80)
print("대한민국 구석구석 사이트의 모든 여행지 정보 수집하기")
print("=" *80)

#query_txt = input('크롤링할 키워드는 무엇입니까?: ')
query_txt = '가을 여행'

f_dir = input("결과 파일을 저장할 폴더명만 쓰세요(없으면 :c:\\doit\\data\\):")

if f_dir == '':
    f_dir = "C:\\doit\\data\\"
    
# 저장될 파일위치와 이름을 자동 지정합니다
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

ff_name = f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fx_name = f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'
fc_name = f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'

#ff_name = input('검색 결과를 저장할 txt 파일경로와 이름을 지정하세요(예:c:\\temp\\test.txt): ')
#fc_name = input('검색 결과를 저장할 csv 파일경로와 이름을 지정하세요(예:c:\\temp\\test.csv): ')
#fx_name = input('검색 결과를 저장할 xls 파일경로와 이름을 지정하세요(예:c:\\temp\\test.xls): ')

##Step 3. 크롬 드라이버를 사용해서 웹 브라우저를 실행합니다.
path = "c:/doit/chromedriver_win32/chromedriver.exe"
driver = webdriver.Chrome(path)

driver.get("https://korean.visitkorea.or.kr/main/main.html")
time.sleep(2)  # 창이 모두 열릴 때 까지 2초 기다립니다.

#코로나 안내 팝업창 있을 경우 닫기 클릭하기 
try :
    driver.find_element_by_xpath('//*[@id="safetyStay1"]/div/div/div/button').click()
except :
    print("안내 팝업창이 없습니다")
    
##Step 4. 검색창의 이름을 찾아서 검색어를 입력하고 검색버튼을 실행합니다. 

#driver.find_element_by_id("btnSearch").click()  ## 검색 디폴트값에 따라 움직이는데 예외처리 방법으로 해결?

driver.find_element_by_id("btnSearch").click()

element = driver.find_element_by_id("inp_search")

try :
    element.send_keys(query_txt)  # 검색 단추가 없어서 에러나면 다시 한번 눌러서 검색창을 띠움
except:
    driver.find_element_by_id("btnSearch").click()
    element = driver.find_element_by_id("inp_search")
    element.send_keys(query_txt)

driver.find_element_by_link_text("검색").click()

# Step 4. 현재 페이지에 있는 내용을 화면에 출력하기

time.sleep(1)

html = driver.page_source   # 검색으로 나온 웹페이지 전체를 읽음
soup = BeautifulSoup(html, 'html.parser')  # 해당 페이지를 BS로 파싱
content_list = soup.find('ul',class_='list_thumType flnon') # 내용 중 해당 부분만 검색해서 리스트로

print(content_list)
print(len(content_list))

# 소주제 1: 특정 항목들을 분리해서 추출하기
no = 1
no2 =[ ]
contents2=[ ]
tags2=[ ]

for i in content_list:
    no2.append(no)
    print('번호:',no)
    
    contents = i.find('div','tit').get_text( )
    contents2.append(contents)
    print('내용:',contents.strip())
    
    tag = i.find('p','tag').get_text()
    tags2.append(tag)
    print('태그:',tag.strip())
    print("\n")
    
    no += 1
    
# 소주제 2: 분리 수집된 데이터를 데이터 프레임으로 만들어서 csv , xls 형식으로 저장하기
# 출력 결과를 txt 파일로 저장하기
f = open(ff_name, 'a',encoding='UTF-8')
f.write(str(contents2))
f.write(str(tags2))
f.close( )
print(" txt 파일 저장 경로: %s" %ff_name)    
    
# 출력 결과를 표(데이터 프레임) 형태로 만들기
import pandas as pd

korea_trip = pd.DataFrame()
korea_trip['번호']=no2
korea_trip['내용']=contents2
korea_trip['태그']=tags2

# csv 형태로 저장하기
korea_trip.to_csv(fc_name,encoding="utf-8-sig")  # csv 파일 인코딩은 'utf-8-sig' 인 것에 유의
print(" csv 파일 저장 경로: %s" %fc_name)

# 엑셀 형태로 저장하기
import xlwt   # pip install xlwt 실행 후 수행
korea_trip.to_excel(fx_name)
print(" xls 파일 저장 경로: %s" %fx_name)

대한민국 구석구석 사이트의 모든 여행지 정보 수집하기
결과 파일을 저장할 폴더명만 쓰세요(없으면 :c:\doit\data\):
<ul class="list_thumType flnon"><li class="bdr_nor"><div class="photo"><a href="javascript:" onclick='goDetail("304","e418f85b-47c1-4931-aac0-7e920b7bc6e0")'><img alt="느릿하거나 우아하거나 열정적인 가을 원주 여행" src="https://cdn.visitkorea.or.kr/img/call?cmd=VIEW&amp;id=ceb7ac33-dac5-4960-928f-46427eff47ee"/></a></div><div class="area_txt"> <div class="tit"> <a href="javascript:" onclick='goDetail("304","e418f85b-47c1-4931-aac0-7e920b7bc6e0")'>느릿하거나 우아하거나 열정적인 <!--HS-->가을<!--HE--> 원주 <!--HS-->여행<!--HE--></a> </div> <p>강원도 원주시</p> <p class="tag"><span>#장애인추천코스</span><span>#원주가볼만한곳</span><span>#원주당일코스</span><span>#치악산금강소나무숲길</span><span>#원주옻문화센터</span><span>#원주드림체육관</span><span>#원주역사박물관</span><span>#원주한지테마파크</span><span>#돼지문화원</span><span>#뮤지엄산</span><span>#간현유원지</span><span>#원주1박2일코스</span><span>#원주2박3일코스</span><span>#무장애여행지</span><span>#장애인추천여행지</span><span>#가을여행</span> </p></div><button class="btn_view" title="내용 더보기" type="button"

In [14]:
# 문자열로 돌려 줄 때

con_1 = ''
for i in content_list:
#    con = i.get_text()
    con = i.text.strip()+'\n'
    con_1 = con_1 + con
#    con_1.append(con)
#    con_1 = content_list.get_text()
print(con_1)

print('\n\n')
# 리스트로 돌려줄 때

con_2 = []
for i in content_list:
    con = i.get_text()
#    con = i.text()
    con_2.append(con)
#    con_1 = content_list.get_text()
print(con_2) 
print(len(con_2))

'''
# 본문 내용 수집하기
contents2=[ ]

for i in content_list:

    print("\n")
    cont = i.get_text()
    contents2.append(cont)
    
print(contents2)'''


단풍이 알록달록 물들어 동네만 걸어도 눈이 즐거운 요즘, 이번 주말엔 단풍여행을 떠나보세요.가을에 가면 딱 좋을 봉화 단풍여행 코스 지금 소개합니다.★추천여행코스 : 국립백두대간수목원 → 분천역 트레킹 → 소강고택(숙박) → 청암정/석천정사(계곡) → 봉화목재문화체험장
태백산맥과 소백산맥을 품고 있는 경북 봉화는 백두산에서 지리산까지 이어지는 우리나라 백두대간의 한 축에 자리 잡은 지역이기도 한데요. 대한민국 생태계의 보고라 할 수 있는 백두대간의 산림생물자원의 보호 및 연구를 위해 이곳 봉화에 대규모의 수목원을 조성해서 2018년에 개장을 했다고 합니다.
특히 우리나라에서는 광릉수목원 다음으로 두 번째로 만들어진 국립수목원이며, 그 규모만 해도 대략 5000ha에 이르러 아시아 최대 규모를 자랑하는 곳이라고 해요.
수목원 규모가 워낙 크고 넓은 곳이다 보니 처음 수목원을 들어가게 되면 어디서부터 어떻게 돌아봐야 할지 길이 많이 헷갈립니다. 그래서 만약 첫 방문이라면, 수목원 입구 방문자 센터에서 직원분에게 시간대별 코스를 추천받아 관람하시면 도움이 됩니다.
대략 트램을 타고 움직이는 트램 코스와, 걸어서 둘러보는 도보코스가 있는데, 시간을 절약하고 중요 포인트만 보시려면 트램을 타는 것도 좋은 방법입니다. 주말과 휴일에는 방문객들이 많아 탑승 대기시간이 길어질 수도 있다는 점 참고하세요.
국립백두대간수목원의 관람코스 중에서 가장 인기가 많고 꼭 들러 볼 만한 곳이 있는데, 바로 백두산 호랑이가 살고 있는 호랑이 숲입니다. 이미 우리나라에서는 사라진 지 100년이 되었고, 현재 멸종 위기종으로 알려진 백두산 호랑이의 종 보전을 위해 호랑이의 자연서식지와 유사한 수목원 내의 넓은 숲을 새로운 보금자리로 만든 곳이라고 하는데요. 현재 3마리가 방사되어 살고 있다고 합니다.
숲이 워낙 넓다 보니 호랑이 숲에 간다고 하더라도 실제 백두산 호랑이를 볼 수 있는 기회는 그리 많지 않다고 하는데, 이날은 운이 좋게도 호랑이를 바로 앞에서 볼 수 있었습니다. 그리고 이곳은 

'\n# 본문 내용 수집하기\ncontents2=[ ]\n\nfor i in content_list:\n\n    print("\n")\n    cont = i.get_text()\n    contents2.append(cont)\n    \nprint(contents2)'

In [49]:
# 추출할 게시글 목록 만들기 

item = []
count = 0

item_list = soup.find('ul','list_thumType flnon').find_all('li','bdr_nor')
#print(item_list)
#print('content_list', len(content_list))
#print('item_list', len(item_list))

for i in item_list :
    item_list_2 = i.find('div','area_txt').find('div','tit')
    print(item_list_2.get_text())
    item.append(item_list_2())
    
print(item)
#print('content_list', len(content_list))


 느릿하거나 우아하거나 열정적인 가을 원주 여행 
 전어·꽃게·대하 가을 별미 삼총사와 함께한 서해안 가을 여행 
 가을은 역시 원주, 원주로 떠나는 가을 여행 원츄! 
 은빛 억새와 황금 들녘, 서해 낙조가 어우러진 보령 오서산 가을 여행 
 중앙선 따라 떠나는 늦가을 간이역 여행 
 한국관광공사 추천 9월 걷기여행길, 초가을의 선선함을 만끽하러 가는 길 
 가까이에서 느껴본 가을 내음, 포천/연천 가을여행 스팟 
 해발 700m 숲의 하룻밤, 이색 체험 태백 가을 여행 
 늦가을 바다의 맛과 풍경이 어우러진 여행, 충남 보령 
 대추처럼 달콤한 충북알프스 가을 여행 
[[<a href="javascript:" onclick='goDetail("304","e418f85b-47c1-4931-aac0-7e920b7bc6e0")'>느릿하거나 우아하거나 열정적인 <!--HS-->가을<!--HE--> 원주 <!--HS-->여행<!--HE--></a>], [<a href="javascript:" onclick='goDetail("304","fe0c20bd-1e43-411d-8368-043582a79a0b")'>전어·꽃게·대하 <!--HS-->가을<!--HE--> 별미 삼총사와 함께한 서해안 <!--HS-->가을<!--HE--> <!--HS-->여행<!--HE--></a>], [<a href="javascript:" onclick='goDetail("304","3d1e9fc4-1f99-490d-a623-24e875bc3f48")'><!--HS-->가을<!--HE-->은 역시 원주, 원주로 떠나는 <!--HS-->가을<!--HE--> <!--HS-->여행<!--HE--> 원츄!</a>], [<a href="javascript:" onclick='goDetail("304","c4878e1a-6029-48c1-8447-2d3601467f3a")'>은빛 억새와 황금 들녘, 서해 낙조가 어우러진 보령 오서산 <!--HS-->가을<!--HE--> <!--HS-->여행<!--HE--